<a href="https://colab.research.google.com/github/KevinHern/SemOpLabs/blob/master/SemOpLabs/Lab3/Laboratorio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [0]:
#Getting imdb dataset

train_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train', 'test'),
    as_supervised=True)

In [0]:
#Splitting the dataset into 50% train and 50% validation using shard

validation_data = train_data.shard(2, 0)
train_data = train_data.shard(2, 1)

In [0]:
train_data, validation_data, test_data

In [0]:
#Show some rows

example_pool, labels = next(iter(train_data.batch(5)))
example_pool, labels


In [0]:
#Importing keras layer

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [0]:
#Building model

model = Sequential([
  hub_layer,
  Flatten(),
  Dense(64, activation='relu'),
  Dense(16, activation='relu'),
  Dense(4, activation='relu'),
  Dense(1, activation='sigmoid')
])


model.summary()

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
#Training model
history = model.fit(train_data.shuffle(10000).batch(256),
                    epochs=10,
                    validation_data=validation_data.batch(256),
                    verbose=1)

#10 epochs leds to overfitting

In [0]:
#Testing model
results = model.evaluate(test_data.batch(100), verbose=1)



In [0]:
testing_pool, test_labels = next(iter(validation_data.shuffle(500).batch(5)))
testing_pool, test_labels

In [0]:
#Predicted label, true label
model.predict(testing_pool), test_labels